In [1]:
from json import dumps, loads
import time
import threading 
import ast
import unittest
import requests
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient
G = setup()[0]


In [2]:
global_client = ChainspaceClient(port=5000)
init_transaction = surge.init()
init_token1 = init_transaction['transaction']['outputs'][0]
init_token2 = init_transaction['transaction']['outputs'][1]
init_token3 = init_transaction['transaction']['outputs'][2]
init_token4 = init_transaction['transaction']['outputs'][3]
init_token5 = init_transaction['transaction']['outputs'][4]
init_token6 = init_transaction['transaction']['outputs'][5]
global_client.process_transaction(init_transaction)

inputs: ()
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 2}", "{\"type\": \"InitToken\", \"location\": 2}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {}}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}


<Response [200]>

In [3]:
# Create prosumer clients for locations 0 and 1
c00 = surge.SurgeClient(5000, init_token1) # location=0
c01 = surge.SurgeClient(5000, init_token2) # location=0
c10 = surge.SurgeClient(5001, init_token3) # location=1
c11 = surge.SurgeClient(5001, init_token4) # location=1
# Create sreps for locations 0 and 1 which will be clients in location 2
rep0 = surge.SREPClient(5002, init_token5) # location=2
rep1 = surge.SREPClient(5002, init_token6) # location=2

# Elect rep1 as SREP for location 0
votes = (c00.cast_srep_vote(rep0.pub), c01.cast_srep_vote(rep0.pub))
rep0.create_srep_client(5000, votes) 

# Elect rep2 as SREP for location 1
votes = (c10.cast_srep_vote(rep1.pub), c11.cast_srep_vote(rep1.pub))
rep1.create_srep_client(5001, votes) 


inputs: ('{"type": "InitToken", "location": 0}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_4b87d3a7997fc917d1ee75144f6ea37fff3814460efa85d87732a781676eda38"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd037d6cfca3a3e2b1f0b4fab15a07946a21aaecff75e5a7912574251455", "92c71d002b17e71b8fa8aa51e1e3ba4540982bb28c4b718ceb2c6c09f67816a2a1c71d002b626f0ceee76f38bf0909da33aad58d5cea2870953b3d1d4b10c010f3"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd037d6cfca3a3e2b1f0b4fab15a07946a21aaecff75e5a7912574251455\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd037d6cfca3a3e2b1f0b4fab15a07946a21aaecff75e5a7912574251455\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd037d6cfca3a3e2b1f0b4fab15a07946a21aaecff75e5a7912574251455\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surg

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
inputs: ('{"type": "VoteSlipToken", "location": 0, "pub": "c7220292cd02c9bd03bd67740f083b629f6f711cfa698cce071ea0d34280cb610fd5d8df10"}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_77c8a0407f92faeef18e5ac91a40739270a53bbf98d7a7a44cdad9483fb8b4dc"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b9205180845f408873abc39509216033168fe1ee5dbb23d22d7687c38c71d002b06c3af22a45e32f344d3554a0b35a4610730ca7420e5107c0db168f4"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd027ca7d71f28ea0e0144658e947a8912c4e1f23d39773ce8e84428b7d4\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd03bd67740f083b629f6f711cfa698cce071ea0d34280cb610fd5d8df10\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03bd67740f083b629f6f711cfa698cce071ea0d34280cb610fd5d8df10\"}"], "returns": [], "dependencies"

In [4]:
t00 = threading.Thread(target=c00.submit_bid, args=('EBBuy',50)) 
t01 = threading.Thread(target=c01.submit_bid, args=('EBSell',55)) 
trep0 = threading.Thread(target=rep0.accept_bids) 

t10 = threading.Thread(target=c10.submit_bid, args=('EBBuy',55)) 
t11 = threading.Thread(target=c11.submit_bid, args=('EBSell',50)) 
trep1 = threading.Thread(target=rep1.accept_bids) 


In [5]:
t00.start()
t01.start()
trep0.start()
t10.start()
t11.start()
trep1.start()

t00.join()
t01.join()
trep0.join()
t10.join()
t11.join()
trep1.join()


inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd037d6cfca3a3e2b1f0b4fab15a07946a21aaecff75e5a7912574251455"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 1, "pub": "c7220292cd02c9bd02819fc35ea413670168692e081d2cba45a241c70d860d2b134dbb5672"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd03bd67740f083b629f6f711cfa698cce071ea0d34280cb610fd5d8df10"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 1, "pub": "c7220292cd02c9bd03f203c566f01833a50de6b41c6dd7571c99d4a164b40976c9e6c629c5"}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_2d9e1d283728e94e927c022cd39c2eb52c5984455cdf9f46e2a9a9ce9f75dd03"], "methodID": "submit_bid_proof", "parameters": ["EBBuy", "92c71d002bfbbc37ab24a6bab65d4445898b9820d765343969b5a7c44607917bf5c71d002ba854c2e24dab8938971605396235ab09bb1e64ef5297817835f6bec1"], "outputs": ["{\"bid_type\": 

inputs: ('{"bid_type": "EBBuy", "type": "BidProof", "location": 1, "pub": "c7220292cd02c9bd02819fc35ea413670168692e081d2cba45a241c70d860d2b134dbb5672", "quantity_sig": "92c71d002bd418f65db6f1398711a2513cfd3e4c5ea909de32e016e783b5ef1697c71d002bd287372eb3d343c09dacb870ca822cd2265a2f74a929b4f7fb26af59"}',)
reference_inputs: ()
inputs: ('{"bid_type": "EBSell", "type": "BidProof", "location": 0, "pub": "c7220292cd02c9bd03bd67740f083b629f6f711cfa698cce071ea0d34280cb610fd5d8df10", "quantity_sig": "92c71d002baeff15f7a8930a879818ea3e4638f8162f82625553298933ad6d6a3cc71d002b633bbc0de48caa1ae5ada1ecd93a16d9ef43869b89ef8cb835d8e328"}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["1_4f514d043314a6f1642fd977e176ffc24b10b54cde4f0daf7c4f7493e892d39a"], "methodID": "submit_bid", "parameters": [55, "92c71d002ba99c0ad710fbe6573db121aa436e7f1dcdaa2e408a68f793ff010a6ac71d002b2e348859e9f0d2bc503c78b17e312d129221da1aae8a2cd0d5fe3a1c"], "o

In [7]:
# '{}|{}'.format(50, loads(r1.ebtoken)['pub'])
# global_client.fix_json("{'type':'EBBuy','location':0")
# global_client = ChainspaceClient(port=5001)
global_client.get_objects({'location':0,'type':'BidAccept'})
# print buy_bids

# print loads(objs[0])['type']
# bid_proofs = rep1.client.get_objects({ 'type':'EBBuy'})
# print init_token1


http://127.0.0.1:5000/api/1.0/objects?status=1
HTTP/1.1 200 OK


['{"total_sell": 55, "type": "BidAccept", "location": 0, "pub": "c7220292cd02c9bd027ca7d71f28ea0e0144658e947a8912c4e1f23d39773ce8e84428b7d4", "total_buy": 50}']